In [1]:
from datetime import datetime, timedelta
from metloom.pointdata import SnotelPointData
import pandas as pd
import cartopy
import geoviews as gv
import geopandas as gpd
import holoviews as hv
import xarray as xr
from metsim import MetSim
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from metpy.units import units
import metpy.calc as mpcalc
import math
import scipy

import sys
sys.path.append('/Users/clintonalden/Documents/Research/summa_work/')
from utils import lw_clr
from utils import forcing_filler as ff

## Use metloom API to pull snotel data

In [40]:
snotel = '515:WA:SNTL' # Harts Pass
start_date = datetime(2014, 10, 1)
start = str(syr)
end = '2015-09-30'
end_date = datetime(2015, 9, 30)
out_name = 'harts_WY15'



start_spinup = start_date - timedelta(days=91)
end_spinup = start_date - timedelta(days=1)
start_spinup

datetime.datetime(2014, 7, 2, 0, 0)

In [8]:
snotel_point = SnotelPointData(snotel, "MyStation")
df = snotel_point.get_hourly_data(
    datetime(2014, 10, 1), datetime(2015, 9, 30),
    [snotel_point.ALLOWED_VARIABLES.PRECIPITATIONACCUM, snotel_point.ALLOWED_VARIABLES.TEMP, 
     snotel_point.ALLOWED_VARIABLES.SWE, snotel_point.ALLOWED_VARIABLES.SNOWDEPTH]
)

# Clean up the dataframe
df.reset_index(inplace=True)

# Rename columns
replace = {'ACCUMULATED PRECIPITATION':'accppt','AIR TEMP':'airtemp', 'datetime':'time'}
df.rename(columns=replace, inplace=True)
df.set_index('time', inplace=True)
print(df)

df.to_csv('../snotel_csvs/'+out_name+'.csv')
df.drop(columns=['site', 'ACCUMULATED PRECIPITATION_units', 'geometry', 'AIR TEMP_units', 'datasource', 
                 'SWE', 'SWE_units', 'SNOWDEPTH', 'SNOWDEPTH_units'], inplace=True)

                                  site  \
time                                     
2014-07-02 08:00:00+00:00  515:WA:SNTL   
2014-07-02 09:00:00+00:00  515:WA:SNTL   
2014-07-02 10:00:00+00:00  515:WA:SNTL   
2014-07-02 11:00:00+00:00  515:WA:SNTL   
2014-07-02 12:00:00+00:00  515:WA:SNTL   
...                                ...   
2015-09-30 04:00:00+00:00  515:WA:SNTL   
2015-09-30 05:00:00+00:00  515:WA:SNTL   
2015-09-30 06:00:00+00:00  515:WA:SNTL   
2015-09-30 07:00:00+00:00  515:WA:SNTL   
2015-09-30 08:00:00+00:00  515:WA:SNTL   

                                                           geometry  accppt  \
time                                                                          
2014-07-02 08:00:00+00:00  POINT Z (-120.65860 48.72047 6490.00000)    41.8   
2014-07-02 09:00:00+00:00  POINT Z (-120.65860 48.72047 6490.00000)    41.8   
2014-07-02 10:00:00+00:00  POINT Z (-120.65860 48.72047 6490.00000)    41.8   
2014-07-02 11:00:00+00:00  POINT Z (-120.65860 48.72047 64

In [9]:
df

,accppt,airtemp
time,,
2014-07-02 08:00:00+00:00,41.8,57.20
2014-07-02 09:00:00+00:00,41.8,56.66
2014-07-02 10:00:00+00:00,41.8,57.02
2014-07-02 11:00:00+00:00,41.8,56.48
2014-07-02 12:00:00+00:00,41.9,56.48
...,...,...
2015-09-30 04:00:00+00:00,41.9,47.12
2015-09-30 05:00:00+00:00,41.9,48.20
2015-09-30 06:00:00+00:00,42.0,47.48


## Fill missing timesteps from snotel data

In [10]:
# Convert the index of the dataframe to a DatetimeIndex
df.index = pd.to_datetime(df.index)

# Create a date range from the first to the last timestep
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')

# Find the missing dates
missing_dates = date_range[~date_range.isin(df.index)]

# Print the missing dates
print(missing_dates)

# Reindex the data DataFrame with the missing dates
# Concatenate the original DataFrame with a DataFrame containing the missing dates
new_df = pd.concat([df, pd.DataFrame(index=missing_dates)], axis=0)

# Sort the new DataFrame by the index
new_df = new_df.sort_index()
df = new_df

# Fill NaNs for every other column
df = df.fillna(np.nan)

# Rename index
df.index.name = 'time'

DatetimeIndex(['2014-09-02 01:00:00+00:00', '2014-09-15 20:00:00+00:00',
               '2014-09-15 21:00:00+00:00', '2014-09-15 22:00:00+00:00',
               '2014-09-15 23:00:00+00:00', '2014-09-16 00:00:00+00:00',
               '2014-09-16 01:00:00+00:00', '2014-09-16 02:00:00+00:00',
               '2014-09-16 03:00:00+00:00', '2014-09-16 04:00:00+00:00',
               '2014-09-16 05:00:00+00:00', '2014-09-16 06:00:00+00:00',
               '2014-09-16 07:00:00+00:00', '2014-09-16 09:00:00+00:00',
               '2014-09-16 10:00:00+00:00', '2014-09-16 11:00:00+00:00',
               '2014-09-16 12:00:00+00:00', '2015-05-06 06:00:00+00:00',
               '2015-05-06 07:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)


/var/folders/db/7c91t8d577n1zsyhth1xc7840000gn/T/ipykernel_35517/369623709.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')


In [11]:
# Create a date range from the first to the last timestep
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')

# Find the missing dates
missing_dates = date_range[~date_range.isin(df.index)]

# Print the missing dates
print(missing_dates)

DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='h')


/var/folders/db/7c91t8d577n1zsyhth1xc7840000gn/T/ipykernel_35517/1354454800.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')


## Unit Conversions

In [12]:
# Covert air temperature to celsius
df['airtemp'] = (df['airtemp'] - 32) * 5.0/9.0

# Convert precipitation to mm
df['accppt'] = df['accppt'] * 25.4

# Convert from geodataframe to dataframe
df = pd.DataFrame(df)


## Split up data into spinup state and desired date range for MetSim

In [45]:
from pytz import UTC
# Interpolate the missing values
df.interpolate(inplace=True)

# Seperate the data into two dataframes, before and after October 1
spinstart = pd.to_datetime('2014-07-03').tz_localize('UTC')
spinend = pd.to_datetime('2014-09-30').tz_localize('UTC')
spinup = df.loc[spinstart:spinend]
data = df.loc[datetime(2014, 10, 1).replace(tzinfo=UTC):]

# Copy the dataframe a2 to a2_copy
data_copy = data.copy()

# Create a mask to identify rows where the index is less than or equal to October 2, 2023
mask = data_copy.index <= datetime(2014, 10, 2).replace(tzinfo=UTC)

# Set the 'precip_accum' column to 0 for rows that satisfy the mask condition
data_copy.loc[mask, 'accppt'] = 0

# Update the value of a2 to the modified copy
data = data_copy

# Calculate the difference between the maximum value of 'precip_accum' and the previous value
spinup['pptrate'] = spinup['accppt'].cummax().diff()
data['pptrate'] = data['accppt'].cummax().diff()

# Drop accppt column
spinup.drop(columns=['accppt'], inplace=True)
data.drop(columns=['accppt'], inplace=True)

## Generate SW from MetSim

In [24]:
# Create empty dataset
dates = pd.date_range('10/01/2014', '09/30/2015')
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )

# We are running only one site, at these coordinates
lats = [48.66]
lons = [-119.84]
elev = 1978 # meters
coords = {'time': dates, 'lat': lats, 'lon': lons}

# Create the initial met data input data structure
met_data = xr.Dataset(coords=coords)

In [25]:
for varname in ['prec', 't_min', 't_max']:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname)

In [26]:
# Resample the data to daily frequency and calculate the maximum and minimum temperatures
tmax_vals = data['airtemp'].resample('D').max()
tmin_vals = data['airtemp'].resample('D').min()

# Calculate the daily precipitation values
prec_vals = data['pptrate'].resample('D').sum()

# Interpolate the temperature values to fill in any missing days
# tmax_vals = tmax_vals.interpolate(method='linear')
# tmin_vals = tmin_vals.interpolate(method='linear')

met_data['prec'].values[:, 0, 0] = prec_vals

# Assign the daily maximum and minimum temperatures to the met_data xarray, converting to Celsius
met_data['t_min'].values[:, 0, 0] = tmin_vals
met_data['t_max'].values[:, 0, 0] = tmax_vals

met_data.to_netcdf('./input/rc_forcing.nc')

In [27]:
# We form the domain in a similar fashion
# First, by creating the data structure
coords = {'lat': lats, 'lon': lons}
domain = xr.Dataset(coords=coords)
domain['elev'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))
domain['mask'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))

# Add the data
domain['elev'][0, 0] = elev
domain['mask'][0, 0] = 1
domain.to_netcdf('./input/rc_domain.nc')

In [28]:
# Finally, we create the state file - the dates are 90 days prior to 
# the MetSim run dates - as usual, create an empty data structure to
# read the data into
dates = pd.date_range('07/03/2014', '09/30/2014')
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )
lats = [48.66]
lons = [-119.84]
elev = 1978 # meters
coords = {'time': dates, 'lat': lats, 'lon': lons}
state = xr.Dataset(coords=coords)
for varname in ['prec', 't_min', 't_max']:
    state[varname] = xr.DataArray(data=np.full(shape, np.nan),
                               coords=coords, dims=dims,
                               name=varname)
    
# Resample precip to daily
prec_vals = spinup['pptrate'].resample('D').sum()

# Resample the data to daily frequency and calculate the maximum and minimum temperatures
tmax_vals = spinup['airtemp'].resample('D').max()
tmin_vals = spinup['airtemp'].resample('D').min()

# Do precip data
state['prec'].values[:, 0, 0] = prec_vals

# And now temp data and convert to C
state['t_min'].values[:, 0, 0] = tmin_vals
state['t_max'].values[:, 0, 0] = tmax_vals
state.to_netcdf('./input/rc_state.nc')

In [29]:
dates = pd.date_range('10/01/2014', '09/30/2015')
params = {
    'time_step'    : "60",       
    'start'        : dates[0],
    'stop'         : dates[-1],
    'forcing'      : './input/rc_forcing.nc',     
    'domain'       : './input/rc_domain.nc',
    'state'        : './input/rc_state.nc',
    'forcing_fmt'  : 'netcdf',
    'out_dir'      : './output',
    'out_prefix': 'salmon',
    'scheduler'    : 'threading',
    'chunks'       : 
        {'lat': 1, 'lon': 1},
    'forcing_vars' : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'state_vars'   : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'domain_vars'  : 
        {'elev': 'elev', 'lat': 'lat', 'lon': 'lon', 'mask': 'mask'}
    }               

ms = MetSim(params)
ms.run()
output = ms.open_output().load()

In [30]:
output

<xarray.Dataset> Size: 280kB
Dimensions:         (time: 8760, lat: 1, lon: 1)
Coordinates:
  * time            (time) datetime64[ns] 70kB 2014-10-01 ... 2015-09-30T23:0...
  * lat             (lat) float64 8B 48.66
  * lon             (lon) float64 8B -119.8
Data variables:
    temp            (time, lat, lon) float32 35kB 2.028 1.497 ... 10.74 10.24
    prec            (time, lat, lon) float32 35kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shortwave       (time, lat, lon) float32 35kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    longwave        (time, lat, lon) float32 35kB 275.9 273.8 ... 307.5 305.4
    vapor_pressure  (time, lat, lon) float32 35kB 0.5618 0.5618 ... 0.9123
    rel_humid       (time, lat, lon) float32 35kB 79.46 82.55 ... 70.71 73.1
Attributes: (12/31)
    conventions:              1.6
    title:                    Output from MetSim
    institution:              University of Washington
    source:                   metsim.py
    history:                  Created: Fri May 10 08:28:14 2024 by clintonalden
    comment:                  no comment at this time
    ...                       ...
    rain_scalar:              0.75
    tday_coef:                0.45
    lapse_rate:               0.0065
    chunks:                   {lat: 1, lon: 1}
    scheduler:                threading
    forcing:                  ./input/rc_forcing.nc

## Create SUMMA forcing netCDF

In [31]:
out_df = output.to_dataframe()
out_df.reset_index(inplace=True)
out_df.set_index('time', inplace=True)

In [32]:
# Remove timezone from index
data.index = data.index.tz_convert(None)

# Convert precipitation rate from m hr^-1 to kg m^-2 s^-1
data['pptrate'] = data['pptrate']/3600

# Generate relative humidity assuming T_d is overnight low temperature
# Used to calculate specific humidity and longwave radiation
ff.fill_rel_hum(data)

# Convert airtemp to Kelvin
data['airtemp'] = (1.03*(data['airtemp']-0.9)) + 273.15 # Currier snotel temp correction

# Generate pressure from hypsometric equation and site elevation (1981m)
ff.fill_pressure(data, elev)

# Generate specific humidity
ff.fill_spec_hum(data)
data['spechum'] = data['spechum'].clip(lower=0.001)


# Set shortwave radiation to MetSim output
data['SWRadAtm'] = out_df['shortwave']

# Generate longwave radiation
data['LWRadAtm'] = lw_clr.dilleyobrien1998(data['airtemp'], data['rh'])

# Can alternatively use the MetSim LW radiation
# data['LWRadAtm'] = out_df['longwave']

# Set wind to 2 m/s
data['windspd'] = 2

# Fill in missing values
data['pptrate'] = data['pptrate'].fillna(0)

# Drop unnecessary columns
data = data.drop(columns=['rh'])

# Interpolate the missing values
data.interpolate(inplace=True)

In [33]:
template = xr.open_dataset('../summa_forcing_template.nc')

In [35]:
# Convert dataframe to xarray
dsx = data.to_xarray()

# Loop through variables and add attributes from template forcing file
for data_var in dsx:
    dsx[data_var].attrs = template[data_var].attrs
    
# Add hru dimension
dsx = dsx.expand_dims(dim={'hru':1})

# Add gap-filled and datastep variables
dsx['gap_filled'] = xr.DataArray(np.ones((1,dsx.time.shape[0])),dims = ['hru','time'])
dsx['data_step'] = 3600 # 3600 seconds for 1hr timesteps

# Transpose gap filled variable to match dimensions with the rest
# dsx['gap_filled'] = dsx['gap_filled'].T

# Convert all to float64
for var in dsx.data_vars:
    dsx[var] = dsx[var].astype(np.float64)

# Set hruID based on template
dsx['hruId'] = (xr.DataArray(np.ones((1))*template['hruId'].values,dims = ['hru'])).astype(np.int32)

# Transpose all variables to match SUMMA dimensions
count = 0
for var in dsx.data_vars:
    print(var,count)
    count += 1
    if count <= 7:
        attribs = dsx[var].attrs
        arr_t = dsx[var].values.T
        dsx[var] = xr.DataArray(dims = ['time','hru'],data = arr_t)
        dsx[var].attrs = attribs

# Set encoding for the time variable
# dsx['time'].encoding = {'_FillValue': np.nan, 'units': 'hours since 1990-01-01', 'calendar': 'proleptic_gregorian'}

# Set hruID based on template
dsx['hruId'] = (xr.DataArray(np.ones((1))*template['hruId'].values,dims = ['hru'])).astype(np.float64).fillna(0).astype(np.int32)

dsx.to_netcdf('../../model/forcings/'+out_name+'.nc',
                        encoding = {"time":
                                        {'dtype' : 'float64',
                                         'units' : 'hours since 1990-01-01 00:00:00',
                                         'calendar' : 'standard'}})

airtemp 0
pptrate 1
airpres 2
spechum 3
SWRadAtm 4
LWRadAtm 5
windspd 6
gap_filled 7
data_step 8
hruId 9


In [36]:
dsx

<xarray.Dataset> Size: 630kB
Dimensions:     (time: 8745, hru: 1)
Coordinates:
  * time        (time) datetime64[ns] 70kB 2014-10-01 ... 2015-09-30T08:00:00
Dimensions without coordinates: hru
Data variables:
    airtemp     (time, hru) float64 70kB 274.4 273.9 273.5 ... 281.1 281.5 281.0
    pptrate     (time, hru) float64 70kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    airpres     (time, hru) float64 70kB 8.696e+04 8.693e+04 ... 8.73e+04
    spechum     (time, hru) float64 70kB 0.004661 0.004516 ... 0.007584 0.007514
    SWRadAtm    (time, hru) float64 70kB 0.0 0.0 0.0 0.0 ... 30.49 142.1 246.6
    LWRadAtm    (time, hru) float64 70kB 240.6 238.3 237.0 ... 275.4 276.9 275.1
    windspd     (time, hru) float64 70kB 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0
    gap_filled  (hru, time) float64 70kB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    data_step   float64 8B 3.6e+03
    hruId       (hru) int32 4B 1001